In [156]:
# 選手情報・過去レース情報から3連単舟券120種をクラス分類する
# LGBMでやってみる。
# めでたく、激しく過学習

# 汎用ライブラリのimport
import sys
import os
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import statsmodels.api as sm
import math
import tensorflow as tf
import collections

import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [119]:
# 自作ライブラリのimport
if os.environ['BR_HOME']+"/boatrace" not in sys.path:
    sys.path.append(os.environ['BR_HOME']+"/boatrace")
#print(sys.path)

from setup.myUtil import dbHandler


In [120]:
# 分析期間の指定は一旦ここでまとめてみる。
trainStartDate="20180101"
trainEndDate="20180731"
# test はtrainからsplitする

In [121]:
dbh=dbHandler.getDBHandle()
#dbHandler.closeDBHandle(dbh)

In [122]:
# trainの元データを取得
with dbh.cursor() as cursor:
    sel_sql = "select * from raceabst_forml_v \
               where raceDate between '%s' and '%s' \
               order by raceId "\
               % (trainStartDate,trainEndDate)
    cursor.execute(sel_sql)
    loadList=cursor.fetchall()
print("traindata:",len(loadList))

traindata: 31824


In [123]:
df = pd.io.json.json_normalize(loadList)
df.head()

,funaken,l1Fcnt,l1boat2r,l1boat3r,l1motor2r,l1motor3r,l1rank,l1score,l2Fcnt,l2boat2r,...,l6boat3r,l6motor2r,l6motor3r,l6rank,l6score,odds,raceDate,raceId,raceWaveHeight,raceWindSpeed
0,2-1-3,1,0.3636,0.5152,0.3558,0.5092,A1,13.871549,0,0.4746,...,0.4545,0.5421,0.6789,B2,0.851669,30.9,2018-01-01,20180101-06-01,3,5.0
1,2-1-6,0,0.2462,0.4462,0.3466,0.4830,B1,3.409304,0,0.3636,...,0.4925,0.3392,0.5088,B1,0.886552,26.2,2018-01-01,20180101-06-02,3,5.0
2,3-5-4,0,0.3692,0.5231,0.3135,0.4108,B2,2.836839,0,0.2714,...,0.4091,0.3472,0.4870,B1,0.580764,99.2,2018-01-01,20180101-06-03,3,5.0
3,1-5-2,0,0.4308,0.6308,0.3676,0.5514,A2,7.488113,0,0.3636,...,0.3939,0.3176,0.4706,B1,0.583266,16.8,2018-01-01,20180101-06-04,3,5.0
4,4-1-3,0,0.3636,0.5152,0.4469,0.5754,B1,6.454111,0,0.3231,...,0.4925,0.3916,0.5663,B2,0.470489,51.5,2018-01-01,20180101-06-05,3,5.0


In [124]:
# 入力のデータ整形
xdf=df.drop(['funaken','odds','raceId','raceDate'],axis=1)
xdf=pd.get_dummies(xdf,columns=['l1rank','l2rank','l3rank','l4rank','l5rank','l6rank'])
xdf.head()


,l1Fcnt,l1boat2r,l1boat3r,l1motor2r,l1motor3r,l1score,l2Fcnt,l2boat2r,l2boat3r,l2motor2r,...,l4rank_B1,l4rank_B2,l5rank_A1,l5rank_A2,l5rank_B1,l5rank_B2,l6rank_A1,l6rank_A2,l6rank_B1,l6rank_B2
0,1,0.3636,0.5152,0.3558,0.5092,13.871549,0,0.4746,0.6610,0.3141,...,1,0,0,0,1,0,0,0,0,1
1,0,0.2462,0.4462,0.3466,0.4830,3.409304,0,0.3636,0.5758,0.3132,...,0,0,0,0,0,1,0,0,1,0
2,0,0.3692,0.5231,0.3135,0.4108,2.836839,0,0.2714,0.5000,0.3333,...,0,0,1,0,0,0,0,0,1,0
3,0,0.4308,0.6308,0.3676,0.5514,7.488113,0,0.3636,0.5303,0.2809,...,0,1,0,0,1,0,0,0,1,0
4,0,0.3636,0.5152,0.4469,0.5754,6.454111,0,0.3231,0.6154,0.3290,...,0,0,1,0,0,0,0,0,0,1


In [125]:
# 結果のOne-Hot表現を作る
ydf=df['funaken']
yLabel = LabelEncoder()
yLabel = yLabel.fit(ydf)
#ydf = pd.DataFrame(yLabel.transform(ydf))
ydf = yLabel.transform(ydf)
#ydf=pd.get_dummies(ydf,columns=['funaken'])
#ydf.head()
#ydf.describe()

In [126]:
# 重み付けのため、オッズのリストを作る
odf=df['odds'].values
#odf=pd.DataFrame(df['odds'])
#odf.describe()
print(type(odf))

<class 'numpy.ndarray'>


In [148]:
X_train, X_test, y_train, y_test,o_train,o_test = train_test_split(xdf, ydf,odf)

In [149]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [181]:
lgbm_params = {
    # 多値分類問題
    'objective': 'multiclass',
    # クラス数は 120
    'num_class': 120,
    #'class_weight':'balanced',
    'random_state':999,
    # 以下、ハイパーパラメタ
    'max_depth':20,
    'num_leaves':3,
    # 正則化
    'reg_alpha':0.1,
    'reg_lambda':0.1,
}


In [182]:
lgb.LGBMClassifier()

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [183]:
model = lgb.train(lgbm_params, lgb_train, valid_sets=lgb_eval)

[1]	valid_0's multi_logloss: 4.6665
[2]	valid_0's multi_logloss: 4.57905
[3]	valid_0's multi_logloss: 4.51022
[4]	valid_0's multi_logloss: 4.45347
[5]	valid_0's multi_logloss: 4.4049
[6]	valid_0's multi_logloss: 4.36305
[7]	valid_0's multi_logloss: 4.32567
[8]	valid_0's multi_logloss: 4.29273
[9]	valid_0's multi_logloss: 4.26329
[10]	valid_0's multi_logloss: 4.23692
[11]	valid_0's multi_logloss: 4.21357
[12]	valid_0's multi_logloss: 4.19212
[13]	valid_0's multi_logloss: 4.17266
[14]	valid_0's multi_logloss: 4.1544
[15]	valid_0's multi_logloss: 4.13827
[16]	valid_0's multi_logloss: 4.12287
[17]	valid_0's multi_logloss: 4.10951
[18]	valid_0's multi_logloss: 4.09693
[19]	valid_0's multi_logloss: 4.08466
[20]	valid_0's multi_logloss: 4.07289
[21]	valid_0's multi_logloss: 4.06252
[22]	valid_0's multi_logloss: 4.05295
[23]	valid_0's multi_logloss: 4.04399
[24]	valid_0's multi_logloss: 4.03553
[25]	valid_0's multi_logloss: 4.02725
[26]	valid_0's multi_logloss: 4.02
[27]	valid_0's multi_loglos

In [195]:
y_pred = model.predict(X_test, num_iteration=model.best_iteration)
y_pred_max = np.argmax(y_pred, axis=1)  # 最尤と判断したクラスの値にする

# 精度 (Accuracy) を計算する
accuracy = sum(y_test == y_pred_max) / len(y_test)
print("accuracy:",accuracy)

# 回収率を計算
res=0
for i in range(len(y_test)):

    if y_test[i]==y_pred_max[i]:
        #print("i:",i,"result:",y_test[i],"forecast:",y_pred_max[i],"forecastProb:",y_pred[i][y_pred_max[i]],"return:",o_test[i],"expect:",y_pred[i][y_pred_max[i]]*o_test[i])
        res += o_test[i] -1
    else:
        #print("i:",i,"result:",y_test[i],"forecast:",y_pred_max[i],"forecastProb:",y_pred[i][y_pred_max[i]],"return:",o_test[i],"expect:",y_pred[i][y_pred_max[i]]*o_test[i])
        pass
print("resultReturn:",res/len(y_test))


accuracy: 0.08810960281548517
i: 0 result: 4 forecast: 0 forecastProb: 0.14681902952790346 return: 5.7 expect: 0.8368684683090497
i: 1 result: 4 forecast: 0 forecastProb: 0.13493464948416256 return: 7.3 expect: 0.9850229412343867
i: 2 result: 21 forecast: 0 forecastProb: 0.14106062894102647 return: 9.5 expect: 1.3400759749397515
i: 3 result: 21 forecast: 1 forecastProb: 0.0824207087397875 return: 20.5 expect: 1.6896245291656438
i: 4 result: 14 forecast: 0 forecastProb: 0.09710743682122291 return: 138.3 expect: 13.42995851237513
i: 5 result: 4 forecast: 7 forecastProb: 0.06507666704926446 return: 6.8 expect: 0.4425213359349983
i: 6 result: 2 forecast: 0 forecastProb: 0.14764959541853204 return: 5.5 expect: 0.8120727748019262
i: 8 result: 1 forecast: 0 forecastProb: 0.11359651390997513 return: 7.3 expect: 0.8292545515428185
i: 9 result: 12 forecast: 7 forecastProb: 0.05970790629542314 return: 36.2 expect: 2.161426207894318
i: 10 result: 2 forecast: 0 forecastProb: 0.11800039944001277 ret

i: 269 result: 7 forecast: 0 forecastProb: 0.11812738151861933 return: 40.3 expect: 4.760533475200359
i: 270 result: 45 forecast: 33 forecastProb: 0.058508264752058 return: 51.7 expect: 3.0248772876813987
i: 271 result: 43 forecast: 4 forecastProb: 0.0428428822677335 return: 18.0 expect: 0.771171880819203
i: 272 result: 101 forecast: 0 forecastProb: 0.0526623034411591 return: 580.9 expect: 30.59153206896932
i: 273 result: 17 forecast: 7 forecastProb: 0.04925938396847946 return: 28.6 expect: 1.4088183814985127
i: 274 result: 13 forecast: 5 forecastProb: 0.10627701800624376 return: 36.7 expect: 3.9003665608291462
i: 275 result: 54 forecast: 40 forecastProb: 0.0768582335585582 return: 15.5 expect: 1.191302620157652
i: 276 result: 42 forecast: 6 forecastProb: 0.0955196502356412 return: 12.3 expect: 1.1748916978983868
i: 277 result: 63 forecast: 0 forecastProb: 0.05941733815078471 return: 70.1 expect: 4.165155404370008
i: 279 result: 89 forecast: 0 forecastProb: 0.08175573211775725 return: 

i: 489 result: 105 forecast: 0 forecastProb: 0.13015935203885104 return: 192.9 expect: 25.107739008294367
i: 490 result: 40 forecast: 0 forecastProb: 0.1133537098035891 return: 26.4 expect: 2.992537938814752
i: 491 result: 16 forecast: 0 forecastProb: 0.17884489011732344 return: 93.4 expect: 16.70411273695801
i: 492 result: 6 forecast: 0 forecastProb: 0.0737969735772086 return: 16.2 expect: 1.1955109719507793
i: 494 result: 23 forecast: 0 forecastProb: 0.1321120795753351 return: 10.2 expect: 1.347543211668418
i: 495 result: 5 forecast: 20 forecastProb: 0.08953555258190356 return: 26.8 expect: 2.3995528091950153
i: 496 result: 111 forecast: 0 forecastProb: 0.09803829552128634 return: 1092.0 expect: 107.05781870924469
i: 497 result: 16 forecast: 2 forecastProb: 0.10204714860779864 return: 115.7 expect: 11.806855093922302
i: 498 result: 33 forecast: 85 forecastProb: 0.03988765937834614 return: 12.9 expect: 0.5145508059806652
i: 499 result: 8 forecast: 0 forecastProb: 0.08652614301397783 r

i: 697 result: 53 forecast: 46 forecastProb: 0.034508198976174714 return: 22.9 expect: 0.7902377565544009
i: 698 result: 61 forecast: 6 forecastProb: 0.08658815821577995 return: 136.5 expect: 11.819283596453962
i: 700 result: 95 forecast: 15 forecastProb: 0.0478093343395369 return: 124.8 expect: 5.966604925574205
i: 701 result: 22 forecast: 86 forecastProb: 0.03281038104393223 return: 60.5 expect: 1.9850280531579
i: 702 result: 68 forecast: 43 forecastProb: 0.05208782217897349 return: 39.2 expect: 2.041842629415761
i: 703 result: 61 forecast: 0 forecastProb: 0.09233330026015915 return: 73.9 expect: 6.823430889225762
i: 705 result: 8 forecast: 5 forecastProb: 0.10434123603917075 return: 17.4 expect: 1.815537507081571
i: 706 result: 19 forecast: 14 forecastProb: 0.06992664244433977 return: 51.0 expect: 3.5662587646613284
i: 707 result: 53 forecast: 2 forecastProb: 0.10967332302578184 return: 375.6 expect: 41.19330012848366
i: 708 result: 8 forecast: 2 forecastProb: 0.10880585354846735 re

i: 976 result: 57 forecast: 5 forecastProb: 0.10683617496265385 return: 217.5 expect: 23.23686805437721
i: 977 result: 9 forecast: 0 forecastProb: 0.14326416206129794 return: 24.0 expect: 3.4383398894711505
i: 978 result: 9 forecast: 0 forecastProb: 0.08088300800851346 return: 35.4 expect: 2.8632584835013764
i: 979 result: 9 forecast: 8 forecastProb: 0.14851179272139378 return: 12.1 expect: 1.7969926919288648
i: 980 result: 32 forecast: 0 forecastProb: 0.05852792945043193 return: 375.1 expect: 21.953826336857016
i: 981 result: 10 forecast: 0 forecastProb: 0.05356264297585745 return: 58.5 expect: 3.133414614087661
i: 982 result: 9 forecast: 1 forecastProb: 0.09322250188955007 return: 16.6 expect: 1.5474935313665312
i: 983 result: 5 forecast: 9 forecastProb: 0.1520487035811742 return: 6.8 expect: 1.0339311843519845
i: 984 result: 26 forecast: 5 forecastProb: 0.05403643413172151 return: 402.7 expect: 21.76047202484425
i: 985 result: 10 forecast: 0 forecastProb: 0.07478904177445193 return:

...

i: 6677 result: 90 forecast: 49 forecastProb: 0.04316941096466205 return: 14.8 expect: 0.6389072822769983
i: 6678 result: 4 forecast: 11 forecastProb: 0.08331720097660582 return: 35.9 expect: 2.9910875150601486
i: 6679 result: 81 forecast: 88 forecastProb: 0.09994038671287846 return: 13.9 expect: 1.3891713753090107
i: 6680 result: 41 forecast: 0 forecastProb: 0.13515266235152543 return: 15.3 expect: 2.067835733978339
i: 6681 result: 32 forecast: 20 forecastProb: 0.08357958820698948 return: 10.4 expect: 0.8692277173526906
i: 6682 result: 8 forecast: 1 forecastProb: 0.13873229799203834 return: 8.7 expect: 1.2069709925307335
i: 6684 result: 6 forecast: 0 forecastProb: 0.051544018543680764 return: 16.7 expect: 0.8607851096794688
i: 6685 result: 10 forecast: 8 forecastProb: 0.06882542349919664 return: 16.0 expect: 1.1012067759871462
i: 6686 result: 76 forecast: 1 forecastProb: 0.15305742032919886 return: 171.6 expect: 26.264653328490525
i: 6687 result: 5 forecast: 6 forecastProb: 0.05764675

i: 6982 result: 6 forecast: 0 forecastProb: 0.13152316774556122 return: 7.2 expect: 0.9469668077680408
i: 6983 result: 15 forecast: 1 forecastProb: 0.09853602894126952 return: 144.8 expect: 14.268016990695827
i: 6984 result: 9 forecast: 4 forecastProb: 0.0556390659674324 return: 49.2 expect: 2.7374420455976742
i: 6985 result: 43 forecast: 13 forecastProb: 0.05812077935340742 return: 91.1 expect: 5.294802999095416
i: 6986 result: 8 forecast: 0 forecastProb: 0.062415721407676546 return: 17.4 expect: 1.0860335524935718
i: 6987 result: 5 forecast: 0 forecastProb: 0.1080730771704494 return: 5.1 expect: 0.5511726935692919
i: 6988 result: 22 forecast: 0 forecastProb: 0.10004517209444717 return: 77.3 expect: 7.733491802900766
i: 6989 result: 1 forecast: 8 forecastProb: 0.09370244723940395 return: 7.4 expect: 0.6933981095715892
i: 6990 result: 17 forecast: 7 forecastProb: 0.14378311485949302 return: 8.9 expect: 1.279669722249488
i: 6991 result: 69 forecast: 4 forecastProb: 0.16083913608921133 r

i: 7307 result: 0 forecast: 4 forecastProb: 0.043606604074149215 return: 51.8 expect: 2.2588220910409293
i: 7308 result: 41 forecast: 5 forecastProb: 0.10481708200520608 return: 19.4 expect: 2.0334513909009977
i: 7309 result: 44 forecast: 6 forecastProb: 0.10220593417152145 return: 133.1 expect: 13.603609838229504
i: 7310 result: 106 forecast: 51 forecastProb: 0.05722824307535713 return: 43.3 expect: 2.4779829251629635
i: 7311 result: 42 forecast: 0 forecastProb: 0.11702125706407941 return: 12.7 expect: 1.4861699647138085
i: 7312 result: 55 forecast: 8 forecastProb: 0.038058800393701304 return: 439.5 expect: 16.726842773031724
i: 7313 result: 5 forecast: 8 forecastProb: 0.08999407548817843 return: 12.6 expect: 1.1339253511510483
i: 7315 result: 4 forecast: 5 forecastProb: 0.07120961249344089 return: 35.1 expect: 2.4994573985197754
i: 7316 result: 40 forecast: 43 forecastProb: 0.04715009721588269 return: 37.5 expect: 1.768128645595601
i: 7317 result: 42 forecast: 1 forecastProb: 0.09849

i: 7582 result: 4 forecast: 3 forecastProb: 0.0682902512533142 return: 23.4 expect: 1.5979918793275523
i: 7583 result: 5 forecast: 4 forecastProb: 0.04239349484245459 return: 16.4 expect: 0.6952533154162552
i: 7584 result: 21 forecast: 0 forecastProb: 0.15407370099163664 return: 73.2 expect: 11.278194912587802
i: 7585 result: 1 forecast: 8 forecastProb: 0.0584948083606937 return: 7.6 expect: 0.4445605435412721
i: 7586 result: 13 forecast: 0 forecastProb: 0.08761385689552648 return: 23.7 expect: 2.0764484084239774
i: 7587 result: 4 forecast: 0 forecastProb: 0.07896583137225636 return: 12.9 expect: 1.0186592247021071
i: 7588 result: 8 forecast: 106 forecastProb: 0.05116243994773819 return: 23.6 expect: 1.2074335827666214
i: 7589 result: 30 forecast: 29 forecastProb: 0.04247386321436706 return: 27.4 expect: 1.1637838520736574
i: 7590 result: 100 forecast: 0 forecastProb: 0.12668160693290437 return: 342.5 expect: 43.38845037451975
i: 7591 result: 24 forecast: 0 forecastProb: 0.059098100240

i: 7815 result: 62 forecast: 1 forecastProb: 0.10751840720191837 return: 68.2 expect: 7.332755371170833
i: 7816 result: 4 forecast: 9 forecastProb: 0.05985790153803762 return: 15.1 expect: 0.903854313224368
i: 7817 result: 6 forecast: 0 forecastProb: 0.0693114674909875 return: 25.6 expect: 1.7743735677692802
i: 7818 result: 38 forecast: 75 forecastProb: 0.06037605934965706 return: 179.2 expect: 10.819389835458544
i: 7819 result: 30 forecast: 7 forecastProb: 0.05765027164101634 return: 338.5 expect: 19.51461695048403
i: 7820 result: 20 forecast: 0 forecastProb: 0.12900838592526873 return: 21.9 expect: 2.825283651763385
i: 7821 result: 10 forecast: 1 forecastProb: 0.1550181189041556 return: 12.6 expect: 1.9532282981923603
i: 7822 result: 82 forecast: 5 forecastProb: 0.0737948847343113 return: 87.4 expect: 6.449672925778808
i: 7823 result: 63 forecast: 7 forecastProb: 0.07314644448334257 return: 194.4 expect: 14.219668807561796
i: 7824 result: 41 forecast: 5 forecastProb: 0.13015957796333

In [196]:
# trainの回収率を計算
y_pred = model.predict(X_train, num_iteration=model.best_iteration)
y_pred_max = np.argmax(y_pred, axis=1)  # 最尤と判断したクラスの値にする

#print(X_train.head())
#print(y_train[0:5])
#print(y_pred_max[0:5])
#c = collections.Counter(y_pred_max)
#print(len(c) )

# 精度 (Accuracy) を計算する
accuracy = sum(y_train == y_pred_max) / len(y_train)
print(accuracy)

# 回収率を計算
res=0
for i in range(len(y_train)):
    if y_train[i]==y_pred_max[i]:
        res += o_train[i] -1
    else:
        pass
print(res/len(y_train))


0.1484833249539132
2.706892073068542
